<a href="https://colab.research.google.com/github/jay2jaykp/ML/blob/master/Online%20Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tqdm
import numpy as np
#from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import numpy as np
from collections import defaultdict
from functools import partial
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure 
import time
import argparse

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [3]:
#upload data
data = pd.read_csv('cheetah.cs.fiu.edu-110108-113008.1.blkparse', header=None)

#split columns
df = pd.DataFrame(data[0].str.split(' ',9).tolist())

#extract one column --> blocktrace sequence
blocktrace = df[3].tolist()
#convert element to int from str
blocktrace = np.array([int(x) for x in blocktrace])

In [4]:
df.head()

,0,1,2,3,4,5,6,7,8
0,89966811921625,4257,nfsd,488846200,8,W,6,0,2c4787c7671c2f638777c4ab72859ce1
1,89967003980393,4253,nfsd,508455704,8,W,6,0,88b93b628d84082186026d9da044f173
2,89967004049935,4253,nfsd,508455712,8,W,6,0,b5e9f4e5ab62a4fff5313a606b0ad4e3
3,89967004093615,4253,nfsd,508455720,8,W,6,0,7af5556ad4d45263bcd8358867333dd4
4,89967324150054,4254,nfsd,508500288,8,W,6,0,88b93b628d84082186026d9da044f173


In [34]:
blocktrace = df[[0,3]]
blocktrace.columns = ['timestamp', 'block']
blocktrace = blocktrace.apply(pd.to_numeric)
blocktrace['timestamp'] = blocktrace['timestamp'] / 1000000000
blocktrace.head()

,timestamp,block
0,89966.811922,488846200
1,89967.003980,508455704
2,89967.004050,508455712
3,89967.004094,508455720
4,89967.324150,508500288


In [6]:
unique_block = blocktrace['block'].unique()
unique_block

array([488846200, 508455704, 508455712, ...,  74832032,  74836592,
          748320], dtype=int64)

In [68]:
#freq = {}
#for i in tqdm(unique_block):
t =blocktrace.loc[blocktrace['block'] == 488846200 ]
t['freq'] = (t['timestamp'].shift(-1) - t['timestamp'])
#total = t['freq'].sum() / t.shape[0]
#freq[i] = total
t.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,timestamp,block,freq
0,89966.811922,488846200,3.203743
3007,89970.015664,488846200,0.447266
3920,89970.462931,488846200,7.024705
7119,89977.487636,488846200,0.404613
7121,89977.892249,488846200,18.042932


In [70]:
t =blocktrace.loc[blocktrace['block'] == 508455712 ]
t['freq'] = (t['timestamp'].shift(-1) - t['timestamp'])
#total = t['freq'].sum() / t.shape[0]
#freq[i] = total
t.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,timestamp,block,freq
2,89967.004050,508455712,2.864533
2705,89969.868583,508455712,0.927258
4584,89970.795841,508455712,1.571987
6994,89972.367828,508455712,1.391236
7023,89973.759064,508455712,0.301394


In [66]:
%timeit blocktrace['freq'] = blocktrace.groupby('block').timestamp.apply(lambda x: x.shift(-1) - x).fillna(0)

KeyboardInterrupt: 

In [67]:
blocktrace

,timestamp,block,freq
0,89966.811922,488846200,3.203743
1,89967.003980,508455704,2.864548
2,89967.004050,508455712,2.864533
3,89967.004094,508455720,2.864532
4,89967.324150,508500288,2.851225
5,89967.324197,508500296,2.851228
6,89967.324240,508500304,2.851228
7,89967.404265,508516672,7.601781
8,89967.404311,508516680,7.601788
9,89967.404359,508516688,7.601789


In [65]:
bl['freq'].unique()

array([0.        , 0.44726644])

In [63]:
bl = blocktrace[3000:4000]

In [19]:
t = blocktrace.loc[blocktrace['block'] == 508455704]
t.head()

,timestamp,block,freq
1,89967003980393,508455704,0.0
2704,89969868528214,508455704,0.0
4583,89970795785915,508455704,0.0
6993,89972367774002,508455704,0.0
7022,89973759010038,508455704,0.0


In [18]:
most_recent_match = blocktrace.query('block == 508455704').shift(1)
most_recent_match

,timestamp,block,freq
1,NaN,NaN,NaN
2704,8.996700e+13,508455704.0,0.0
4583,8.996987e+13,508455704.0,0.0
6993,8.997080e+13,508455704.0,0.0
7022,8.997237e+13,508455704.0,0.0
7034,8.997376e+13,508455704.0,0.0
7041,8.997406e+13,508455704.0,0.0
7050,8.997430e+13,508455704.0,0.0
7056,8.997452e+13,508455704.0,0.0
7062,8.997468e+13,508455704.0,0.0


In [11]:
t = blocktrace
t['freq'] = 0
t['freq'] = np.where(t['block']==488846200, (t['timestamp'].shift(-1) - t['timestamp']) / 1000000000, t['freq'] )
t.head()

,timestamp,block,freq
0,89966811921625,488846200,0.192059
1,89967003980393,508455704,0.000000
2,89967004049935,508455712,0.000000
3,89967004093615,508455720,0.000000
4,89967324150054,508500288,0.000000


In [44]:
t =blocktrace.loc[blocktrace['block'] == 488846200 ]
t['freq'] = (t['timestamp'].shift(-1) - t['timestamp']) / 1000000000
t.head()

C:\Users\Jay\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,timestamp,block,freq
0,89966811921625,488846200,3.203743
3007,89970015664252,488846200,0.447266
3920,89970462930696,488846200,7.024705
7119,89977487635675,488846200,0.404613
7121,89977892249173,488846200,18.042932


In [47]:
total = t['freq'].sum() / t.shape[0]
total

31.08903011429535

In [46]:
t.shape[0]

237

In [20]:
t[0] = t[0].apply(pd.to_numeric)

C:\Users\Jay\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [28]:
t['freq'] = t[0].shift(-1) - t[0]

C:\Users\Jay\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [34]:
t.head()

,0,1,2,3,4,5,6,7,8,freq
0,89966811921625,4257,nfsd,488846200,8,W,6,0,2c4787c7671c2f638777c4ab72859ce1,7.770000e+02
3007,89970015664252,4254,nfsd,488846200,8,W,6,0,56dfdc25ec136fbce64f7fe7e336b9ff,4.472664e+08
3920,89970462930696,4253,nfsd,488846200,8,W,6,0,684fb98a8bd827a7ff80a930255fe6da,7.024705e+09
7119,89977487635675,4250,nfsd,488846200,8,W,6,0,56dfdc25ec136fbce64f7fe7e336b9ff,4.046135e+08
7121,89977892249173,4250,nfsd,488846200,8,W,6,0,684fb98a8bd827a7ff80a930255fe6da,1.804293e+10


In [23]:
from datetime import datetime

In [31]:
df[3].unique().shape

(671728,)

In [33]:
t['freq'][0] = 777

C:\Users\Jay\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Jay\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2961: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
